In [284]:
# !pip install ortools

In [285]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


In [286]:
class Plan:
  def __init__(self):
    self.path = []
    self.distance = None
    self.load = None
    self.output = {}
    self.output['path'] = []
    

In [287]:
almtvsk_kams_coords = [(54.902532, 52.289729), # 0 - Белоглазова 131
                     (54.900249, 52.287286), # 1 - Белоглазова 151
                     (54.905891, 52.270927), # 3 - Гафиатуллина 29Б
                     (54.906383, 52.266427), # 4 - улица Гафиатуллина, 39
                     (54.905829, 52.264387), # 5 - улица Гафиатуллина, 45
                     (54.906518, 52.263822), # 6 - улица Гафиатуллина, 47
                     (54.906518, 52.263822), # 7 - улица Гафиатуллина, 47
                     (54.900078, 52.290205), # 10 - улица Ленина, 66
                     (54.898670, 52.285722), # 11 - улица Ленина, 90
                     (54.896071, 52.288390), # 12 - улица Шевченко, 80
                     (54.900777, 52.267460), # 14 - проспект Строителей, 20Б
                     (54.902294, 52.270047), # 15 - проспект Строителей, 20(тут фото с камеры обрезано, мб это не тот дом)
]

In [288]:
almtvsk_demands = [0, 5,3,6,8,4,9,5,7,3,6,16]

In [289]:
almtvsk_data = {}
almtvsk_data['num_vehicles'] = 4
almtvsk_data['depot'] = 0
almtvsk_data['demands'] = almtvsk_demands
almtvsk_data['vehicle_capacities'] = [15, 15, 15, 15]
almtvsk_data

{'demands': [0, 5, 3, 6, 8, 4, 9, 5, 7, 3, 6, 16],
 'depot': 0,
 'num_vehicles': 4,
 'vehicle_capacities': [15, 15, 15, 15]}

In [290]:
def print_solution(almtvsk_data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    # Display dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
    print(dropped_nodes)
    total_distance = 0
    total_load = 0
    all_plans = []
    output_constants = ['lat', 'lon']
    for vehicle_id in range(almtvsk_data['num_vehicles']):
        plan = Plan()
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += almtvsk_data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            plan.path.append((node_index, route_load))
            plan.output['path'].append({output_constants[i]: almtvsk_kams_coords[node_index][i] for i in (0,1)})
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        plan.path.append((manager.IndexToNode(index), route_load))
        plan.output['path'].append({output_constants[i]: almtvsk_kams_coords[manager.IndexToNode(index)][i] for i in (0,1)})
        plan.distance = route_distance
        plan.load = route_load
        all_plans.append(plan)
        for i,res in enumerate(all_plans):
          all_plans[i].output['path'] = str(res.output['path']).replace('},',"};").replace(']',"").replace('[',"").replace(" ","")
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    return all_plans

In [291]:
import numpy as np
def form_distance_matrix(coordinates):
    X = np.array(coordinates)
    distance_matrix = np.zeros((len(coordinates), len(coordinates)))
    # distance_matrix = np.sqrt(np.sum((X[:,np.newaxis]-X)**2,axis=2)) # эвклидово растояние
    distance_matrix = np.sum(abs(X[:,np.newaxis]-X),axis=2) # Манхэтанская
    return distance_matrix

In [292]:
def np_to_list(arr):
  return [[arr[i,j] for j in range(len(arr[0]))] for i in range(len(arr))]

In [293]:
z = {1:0, 5:6}
del z[1]
z

{5: 6}

In [294]:
def solver():
  def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return almtvsk_data['demands'][from_node]

  def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return almtvsk_data['distance_matrix'][from_node][to_node]

    # Instantiate the data problem.
  almtvsk_data['distance_matrix'] = np_to_list(form_distance_matrix(almtvsk_kams_coords))


    # Create the routing index manager.
  manager = pywrapcp.RoutingIndexManager(len(almtvsk_data['distance_matrix']),
                                           almtvsk_data['num_vehicles'], almtvsk_data['depot'])

    # Create Routing Model.
  routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
  demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
  routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        almtvsk_data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
  # Allow to drop nodes.
  penalty = 1000
  for node in range(1, len(almtvsk_data['distance_matrix'])):
      routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Setting first solution heuristic.
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
  search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
  search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
  solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
  if solution:
      return print_solution(almtvsk_data, manager, routing, solution)
  else:
    print("No solution found")

Sample text

In [295]:
almtvsk_result = solver()


Objective: 1000
Dropped nodes: 11
Route for vehicle 0:
 0 Load(0) ->  4 Load(8) ->  1 Load(13) ->  0 Load(13)
Distance of the route: 0m
Load of the route: 13

Route for vehicle 1:
 0 Load(0) ->  6 Load(9) ->  3 Load(15) ->  0 Load(15)
Distance of the route: 0m
Load of the route: 15

Route for vehicle 2:
 0 Load(0) ->  8 Load(7) ->  5 Load(11) ->  2 Load(14) ->  0 Load(14)
Distance of the route: 0m
Load of the route: 14

Route for vehicle 3:
 0 Load(0) ->  10 Load(6) ->  9 Load(9) ->  7 Load(14) ->  0 Load(14)
Distance of the route: 0m
Load of the route: 14

Total distance of all routes: 0m
Total load of all routes: 56


In [296]:
for res in almtvsk_result:
  print(res.output)
  print(res.distance)
  print(res.load)
  print(res.path)


{'path': "{'lat':54.902532,'lon':52.289729};{'lat':54.905829,'lon':52.264387};{'lat':54.900249,'lon':52.287286};{'lat':54.902532,'lon':52.289729}"}
0
13
[(0, 0), (4, 8), (1, 13), (0, 13)]
{'path': "{'lat':54.902532,'lon':52.289729};{'lat':54.906518,'lon':52.263822};{'lat':54.906383,'lon':52.266427};{'lat':54.902532,'lon':52.289729}"}
0
15
[(0, 0), (6, 9), (3, 15), (0, 15)]
{'path': "{'lat':54.902532,'lon':52.289729};{'lat':54.89867,'lon':52.285722};{'lat':54.906518,'lon':52.263822};{'lat':54.905891,'lon':52.270927};{'lat':54.902532,'lon':52.289729}"}
0
14
[(0, 0), (8, 7), (5, 11), (2, 14), (0, 14)]
{'path': "{'lat':54.902532,'lon':52.289729};{'lat':54.900777,'lon':52.26746};{'lat':54.896071,'lon':52.28839};{'lat':54.900078,'lon':52.290205};{'lat':54.902532,'lon':52.289729}"}
0
14
[(0, 0), (10, 6), (9, 9), (7, 14), (0, 14)]


In [297]:
main()

0